In [1]:
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from scipy.linalg import cho_factor, cho_solve

In [2]:
A = np.load("A_matrix.npy")
B = np.load("B_matrix.npy")
c = np.load("C_matrix.npy")

In [3]:
A1 = np.eye(np.shape(A)[0], np.shape(A)[1])
A = np.hstack([A, A1])
print(np.shape(A))

(6590, 12578)


In [4]:
C = np.hstack([c, np.zeros(len(c))])

In [5]:
At = np.transpose(A)
AAt = np.dot(A, At)
x = np.dot(np.linalg.inv(AAt), B)

In [6]:
print(x)

[-3.85809603e-06 -1.94884957e-06  1.40604686e-07 ... -9.40691579e-10
 -9.43727086e-09 -2.22502953e-09]


In [7]:
x = np.dot(At, x)
print(x)

[-1.12402705e-10  1.12402705e-10  4.30743739e-10 ...  3.56424494e-09
 -3.56424494e-09  4.81087192e-09]


In [8]:
y = np.dot(A, C)
y = np.dot(np.linalg.inv(AAt),(y))
print(np.shape(y))

(6590,)


In [9]:
s = C - np.dot(At, y)

In [10]:
print(np.shape(s))

(12578,)


In [11]:
delta_x = 0.0
delta_s = 0.0

for i in range(0, len(x)):
    delta_x = max(delta_x, -1.5*x[i])
for i in range(0, len(s)):
    delta_s = max(delta_s, -1.5*s[i])
print(delta_x, delta_s)

0.00013700787388288234 2.500006019874031e-06


In [12]:
for i in range(0, len(x)):
    x[i] = x[i] + delta_x
for i in range(0, len(s)):
    s[i] = s[i] + delta_s

o = np.ones(len(s))
i_x = np.dot(x, s)
i_y = np.dot(o, s)
i_z = np.dot(o, x)
delta_x = 0.5 * (i_x/i_y)
delta_s = 0.5 * (i_x/i_z)
print(delta_x, delta_s)

6.850393694352647e-05 0.011967693221543743


In [13]:
mid = B - np.dot(A, x) 
print(min(mid))

-0.00013700787388288237


In [14]:
k = 0
for i in range(0, len(s)):
    if (s[i] < 0):
        k = k + 1
print(k)

0


In [15]:
for i in range(0, len(x)):
    x[i] = x[i] + delta_x
    s[i] = s[i] + delta_s

In [16]:
k = 0
for i in range(0, len(s)):
    if (s[i] < 0):
        k = k + 1
print(k)

0


In [17]:
m = 1000
mx = 1000
for i in range(0, len(x)):
    m = min(m, x[i])
for i in range(0, len(x)):
    mx = min(mx, s[i])
print(m)

0.00011417322823782059


In [18]:
print(mx)

0.0119685265568837


In [19]:
k = 0
for i in range(0, len(s)):
    if (s[i] <= 0):
        k = k + 1
        

In [20]:
print(k)

0


In [21]:
mid = B - np.dot(A, x) 

In [22]:
print(np.shape(x), max(mid))

(12578,) 0.00041102362165281765


In [23]:
k = 0
m = 0
ma = 0
for i in range(0, len(mid)):
    if (mid[i] < 0):
        k = k + 1
        m = min(m, mid[i])
    ma = max(ma, mid[i])
print(k, m, ma)

3295 -0.00020551181082640885 0.00041102362165281765


In [ ]:
result = linprog(C, A_eq = A, b_eq=B, method='interior-point', options={'maxiter':10})['x']

In [ ]:
print(max(result), max(x))

In [18]:
A = np.array([[0.06626872, 0.49782317, 0.16945034],[0.27174321, 0.22702761, 0.35243303]])
B = np.array([0.2964, 0.3061])
C = np.array([0.64, 0.84, 0.38])
x = np.array([0.436, 0.596, 0.554])
s = np.array([0.7355, 0.5293, 0.2713])
y = np.array([1.08, 1.1877])
# print(np.shape(A))
At = np.transpose(A)

In [19]:
n = len(x)
X = np.zeros((n, n))
S = np.zeros((n, n))

for i in range(0, 5):
    for j in range(0, n):
        X[j][j] = x[j]
        S[j][j] = s[j]
    mu = np.dot(np.transpose(x), s)
    mu = mu/n
    
    print(mu)
    rd = C - s - np.dot(np.transpose(A), y)
#     print(rd)
    rp = B - np.dot(A, x)
#     print(rp)
    rc = np.zeros(n)
    for j in range(0, n):
        rc[j] = -1.0*x[j]*s[j]
    S_1 = np.linalg.inv(S)
    D = S_1*X
    ADA = np.dot(A, np.dot(D, np.transpose(A)))
    print(np.dot(A, D))
    AS = np.dot(A, S_1)
    #dy_aff = -1.0 * np.dot(np.linalg.inv(ADA), np.dot(AS, rc) - np.dot(A, np.dot(D, rd)) - rp)
    intm = np.dot(AS, rc) - np.dot(A, np.dot(D, rd)) - rp
    print(max(intm))
    chh, low = cho_factor(ADA)
    dy_aff = cho_solve((chh, low), intm)
    dy_aff = -1.0 * dy_aff
    ds_aff = -1.0 * np.dot(At, dy_aff) + rd
    dx_aff = np.dot(S_1, rc) - np.dot(D, ds_aff)
    
    ap_aff = 1.0
    ad_aff = 1.0
    
    for j in range(0, n):
        if (dx_aff[j] < 0):
            ap_aff = min(ap_aff, -0.9*x[j]/dx_aff[j])
        if (ds_aff[j] < 0):
            ad_aff = min(ad_aff, -0.9*s[j]/ds_aff[j])
    
    x = x + ap_aff * dx_aff
    y = y + ad_aff * dy_aff
    s = s + ad_aff * ds_aff
    
    print("Cost: ", np.dot(np.transpose(C), x))
    print("F: ", min(B - np.dot(A, x)))

0.26214699999999996
[[0.0392837  0.5605566  0.34602097]
 [0.16108775 0.25563661 0.71967526]]
0.2544719659982102
Cost:  0.6332406893982662
F:  -5.551115123125783e-17
0.053936457437246045
[[0.0245228  1.25223098 2.05386224]
 [0.10055882 0.57106825 4.27174647]]
-0.2964
Cost:  0.5660386625465845
F:  -5.551115123125783e-17
0.010650109853263137
[[2.52650500e-03 3.35167559e+00 2.62730607e+01]
 [1.03602511e-02 1.52850037e+00 5.46442952e+01]]
-0.2963999999999996
Cost:  0.5593184598614165
F:  -2.220446049250313e-16
0.001270699077112981
[[2.49692209e-04 2.60197964e+01 2.68465046e+02]
 [1.02389427e-03 1.18660853e+01 5.58369782e+02]]
-0.2964
Cost:  0.5586464395928992
F:  1.1102230246251565e-16
0.00012969012193581663
[[2.49253042e-05 2.52982999e+02 2.69038489e+03]
 [1.02209340e-04 1.15370535e+02 5.59562465e+03]]
-0.29640000000020555
Cost:  0.5585792375660474
F:  -1.1102230246251565e-16


In [ ]:
print(max(abs(result - x)), max(x))

In [ ]:
print(min(s))

In [ ]:
np.savetxt("AAt.csv", AAt, delimiter = " ")

In [ ]:
np.savetxt("Atxt.csv", A, delimiter = " ")
np.savetxt("Btxt.csv", B, delimiter = " ")
np.savetxt("Ctxt.csv", C, delimiter = " ")